In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
from tensorflow import keras

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import wandb
from wandb.integration.keras import WandbCallback

# Load the data
wandb.login()
data = pd.read_csv("./MIMIC_data.csv")


/Users/shaicahana/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
2024-10-26 17:20:29.922831: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: shaicahana (shaicahana-hebrew-university-of-jerusalem). Use `wandb login --relogin` to 

In [2]:

from sklearn.utils import resample
# data_clean = data.dropna()
# y = data_clean['outcome']
# X = data_clean.drop(columns='outcome')
def balance_dataset(df, column, method='undersample', random_state=42):
    """
    Balance a dataset based on the values in a specified column.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame
    column (str): Column name to balance by
    method (str): 'undersample' or 'oversample'
    random_state (int): Random state for reproducibility
    
    Returns:
    pandas.DataFrame: Balanced DataFrame
    """
    # Get value counts
    value_counts = df[column].value_counts()
    print(f"Original class distribution:\n{value_counts}\n")
    
    if method == 'undersample':
        # Get the minimum class count
        min_count = value_counts.min()
        
        # Create balanced dataframe
        balanced_df = pd.concat([
            resample(df[df[column] == val],
                    replace=False,
                    n_samples=min_count,
                    random_state=random_state)
            for val in value_counts.index
        ])
        
    elif method == 'oversample':
        # Get the maximum class count
        max_count = value_counts.max()
        
        # Create balanced dataframe
        balanced_df = pd.concat([
            resample(df[df[column] == val],
                    replace=True,
                    n_samples=max_count,
                    random_state=random_state)
            if count < max_count else df[df[column] == val]
            for val, count in value_counts.items()
        ])
    
    print(f"Balanced class distribution:\n{balanced_df[column].value_counts()}")
    return balanced_df.reset_index(drop=True)



balanced_data_under = balance_dataset(data, 'outcome', method='undersample', random_state=42)
balanced_data_over = balance_dataset(data, 'outcome', method='oversample', random_state=42)

print(data.shape)
print(balanced_data_under.shape)
print(balanced_data_over.shape)

data = balanced_data_over

Original class distribution:
outcome
0.0    1017
1.0     159
Name: count, dtype: int64



NameError: name 'resample' is not defined

In [ ]:


# build input pipeline using tf.data


def get_train_and_val(data_feaure_and_prediction, BATCH_SIZE = 64):
    """_summary_
    this function gets the relevant feaure and variables 
    Args:
        data_feaure_and_prediction (_type_): the last colum should be the output, the first ones should be the feature(s)
        BATCH_SIZE (int, optional): _description_. Defaults to 64.

    Returns:
        _type_: _description_
    """
    # data_feaure_and_prediction = data_feaure_and_prediction.dropna() # drop the irrelevant values
    X = data_feaure_and_prediction[data_feaure_and_prediction.keys()[:-1]] # the feature
    y = data_feaure_and_prediction[data_feaure_and_prediction.keys()[-1]] # its prediction

    X_standard = StandardScaler()
    X_standard = X_standard.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_standard, y, test_size=0.2, random_state=42)

    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

    val_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
    val_dataset = val_dataset.batch(BATCH_SIZE)
    return [train_dataset, val_dataset]


In [ ]:
def make_model(feature_num, input_name="input name"):
    inputs = keras.Input((feature_num,), name=input_name)
    x1 = keras.layers.Dense(64, activation="relu")(inputs)
    x1 = keras.layers.Dropout(0.3)(x1)

    x2 = keras.layers.Dense(64, activation="relu")(x1)
    x2 = keras.layers.Dropout(0.3)(x2)

    outputs = keras.layers.Dense(10, name="predictions")(x2)

    return keras.Model(inputs=inputs, outputs=outputs)



In [ ]:
def train_step(x, y, model, optimizer, loss_fn, train_acc_metric):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)

    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(y, logits) # update the weights of the network

    return loss_value

def test_step(x, y, model, loss_fn, val_acc_metric):
    val_logits = model(x, training=False)
    loss_value = loss_fn(y, val_logits)
    val_acc_metric.update_state(y, val_logits)

    return loss_value

def train(train_dataset, val_dataset,  model, optimizer,
          train_acc_metric, val_acc_metric, loss_fn,
          epochs=10, log_step=200, val_log_step=50):
  
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))

        train_loss = []   
        val_loss = []

        # Iterate over the batches of the dataset
        for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
            loss_value = train_step(x_batch_train, y_batch_train, 
                                    model, optimizer, 
                                    loss_fn, train_acc_metric)
            train_loss.append(float(loss_value))

        # Run a validation loop at the end of each epoch
        for step, (x_batch_val, y_batch_val) in enumerate(val_dataset):
            val_loss_value = test_step(x_batch_val, y_batch_val, 
                                       model, loss_fn, 
                                       val_acc_metric)
            val_loss.append(float(val_loss_value))
            
        # Display metrics at the end of each epoch
        train_acc = train_acc_metric.result()
        print("Training acc over epoch: %.4f" % (float(train_acc),))

        val_acc = val_acc_metric.result()
        print("Validation acc: %.4f" % (float(val_acc),))

        # Reset metrics at the end of each epoch
        train_acc_metric.reset_state()
        val_acc_metric.reset_state()

        # ⭐: log metrics using wandb.log
        wandb.log({'epochs': epoch,
                   'loss': np.mean(train_loss),
                   'acc': float(train_acc), 
                   'val_loss': np.mean(val_loss),
                   'val_acc':float(val_acc)})

In [ ]:
# # initialize wandb with your project name and optionally with configutations.
# # play around with the config values and see the result on your wandb dashboard.
# feature = ["age"]

# config = {
#             "epochs": 10,
#             "batch_size": 32,
#             "log_step": 200,
#             "val_log_step": 50,
#             "architecture": "CNN",
#             "dataset": "MIMIC"
#     }

# run = wandb.init(project='my-tf-integration', config=config)
# config = wandb.config

# prediction = "outcome"

# model_data = data[feature+[prediction]].dropna()
# [train_dataset, val_dataset] = get_train_and_val(model_data)


# # Initialize model.
# model = make_model(len(feature))

# # Instantiate an optimizer to train the model.
# optimizer = keras.optimizers.Adam()
# # Instantiate a loss function.
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# # Prepare the metrics.
# train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
# val_acc_metric = keras.metrics.SparseCategoricalAccuracy()


# train(train_dataset,
#     val_dataset, 
#     model,
#     optimizer,
#     train_acc_metric,
#     val_acc_metric,
#     epochs=config.epochs, 
#     log_step=config.log_step, 
#     val_log_step=config.val_log_step)

# run.finish()  # In Jupyter/Colab, let us know you're finished!




In [ ]:
# initialize wandb with your project name and optionally with configutations.
# play around with the config values and see the result on your wandb dashboard.

def run_with_feature(feature):
      config = {
            "epochs": 20,
            "batch_size": 32,
            "log_step": 200,
            "val_log_step": 50,
            "feature_used": str(feature[0])
      }

      run = wandb.init(project='my-tf-integration', config=config)
      config = wandb.config

      prediction = "outcome"

      model_data = data[feature+[prediction]].dropna()
      [train_dataset, val_dataset] = get_train_and_val(model_data)


      # Initialize model.
      model = make_model(len(feature))

      # Instantiate an optimizer to train the model.
      optimizer = keras.optimizers.Adam()
      # Instantiate a loss function.
      loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

      # Prepare the metrics.
      train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
      val_acc_metric = keras.metrics.SparseCategoricalAccuracy()


      train(train_dataset,
            val_dataset, 
            model,
            optimizer,
            train_acc_metric,
            val_acc_metric,
            loss_fn,
            epochs=config.epochs, 
            log_step=config.log_step, 
            val_log_step=config.val_log_step)

      run.finish()  # In Jupyter/Colab, let us know you're finished!


# feature = ["age"]
# run_with_feature(feature)

In [ ]:
data_keys = data.drop(columns=['outcome']).keys()
len(data_keys)
# data_keys = data_keys[ 

train_acc_dict = dict.fromkeys(data_keys, 0)
val_acc_dict = dict.fromkeys(data_keys, 0)

for elem in data_keys:
    # try:
    run_with_feature([elem])
    #     train_acc_dict[elem] = train_acc_metric
    #     val_acc_dict[elem] = val_acc_dict
    # except:
    #     pass


Start of epoch 0
Training acc over epoch: 0.3691
Validation acc: 0.8305

Start of epoch 1
Training acc over epoch: 0.8681
Validation acc: 0.8305

Start of epoch 2
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 3
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 4
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 5
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 6
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 7
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 8
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 9
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 10
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 11
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 12
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 13
Training acc over epoch: 0.87

acc,▁███████████████████
epochs,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▆▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.8734
epochs,19
loss,0.38691
val_acc,0.83051
val_loss,0.46417



Start of epoch 0
Training acc over epoch: 0.3894
Validation acc: 0.8305

Start of epoch 1
Training acc over epoch: 0.8628
Validation acc: 0.8305

Start of epoch 2
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 3
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 4
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 5
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 6
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 7
Training acc over epoch: 0.8723
Validation acc: 0.8305

Start of epoch 8
Training acc over epoch: 0.8745
Validation acc: 0.8305

Start of epoch 9
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 10
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 11
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 12
Training acc over epoch: 0.8723
Validation acc: 0.8305

Start of epoch 13
Training acc over epoch: 0.87

acc,▁███████████████████
epochs,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▇▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.8734
epochs,19
loss,0.39419
val_acc,0.83051
val_loss,0.46276



Start of epoch 0
Training acc over epoch: 0.4298
Validation acc: 0.8305

Start of epoch 1
Training acc over epoch: 0.8585
Validation acc: 0.8305

Start of epoch 2
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 3
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 4
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 5
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 6
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 7
Training acc over epoch: 0.8723
Validation acc: 0.8305

Start of epoch 8
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 9
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 10
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 11
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 12
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 13
Training acc over epoch: 0.87

...

acc,▁▇██████████████████
epochs,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▇▅▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█
val_loss,█▆▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.84169
epochs,19
loss,0.44047
val_acc,0.86316
val_loss,0.38871



Start of epoch 0
Training acc over epoch: 0.2224
Validation acc: 0.8249

Start of epoch 1
Training acc over epoch: 0.7507
Validation acc: 0.8362

Start of epoch 2
Training acc over epoch: 0.8357
Validation acc: 0.8362

Start of epoch 3
Training acc over epoch: 0.8371
Validation acc: 0.8362

Start of epoch 4
Training acc over epoch: 0.8371
Validation acc: 0.8362

Start of epoch 5
Training acc over epoch: 0.8371
Validation acc: 0.8362

Start of epoch 6
Training acc over epoch: 0.8371
Validation acc: 0.8362

Start of epoch 7
Training acc over epoch: 0.8371
Validation acc: 0.8362

Start of epoch 8
Training acc over epoch: 0.8371
Validation acc: 0.8362

Start of epoch 9
Training acc over epoch: 0.8357
Validation acc: 0.8362

Start of epoch 10
Training acc over epoch: 0.8371
Validation acc: 0.8362

Start of epoch 11
Training acc over epoch: 0.8371
Validation acc: 0.8362

Start of epoch 12
Training acc over epoch: 0.8357
Validation acc: 0.8362

Start of epoch 13
Training acc over epoch: 0.83

acc,▁▇██████████████████
epochs,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▇▅▄▃▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁
val_acc,▁███████████████████
val_loss,█▇▅▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.83711
epochs,19
loss,0.44361
val_acc,0.83616
val_loss,0.45862



Start of epoch 0
Training acc over epoch: 0.7362
Validation acc: 0.8305

Start of epoch 1
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 2
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 3
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 4
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 5
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 6
Training acc over epoch: 0.8723
Validation acc: 0.8305

Start of epoch 7
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 8
Training acc over epoch: 0.8713
Validation acc: 0.8305

Start of epoch 9
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 10
Training acc over epoch: 0.8723
Validation acc: 0.8305

Start of epoch 11
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 12
Training acc over epoch: 0.8734
Validation acc: 0.8305

Start of epoch 13
Training acc over epoch: 0.87

acc,▁███████████████████
epochs,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▆▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.8734
epochs,19
loss,0.4079
val_acc,0.83051
val_loss,0.46185
